In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import sys

sys.path.append(os.path.abspath("../"))
from dataset import DS_NAMES
from dataset import DS_DIMENSIONS
from utilities.figure_tools import get_latest_results_directory, make_fig_directory, load_results

results_path = "../results/turing/vary_enum.csv"
output_dir = "./vary_enum_tur/"
export_figures = False

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)


In [13]:
all_results = pd.read_csv(results_path).groupby(["dataset", "explainer", "hard_voting", "enumeration"]).mean().reset_index()
print("datasets", list(all_results["dataset"].unique()))
all_results.head()

datasets ['cancer', 'glass', 'magic', 'spambase', 'vertebral']


,dataset,explainer,hard_voting,enumeration,n_trees,max_depth,iteration,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,FACETIndex,False,Axes,100.0,NaN,4.5,0.947611,0.926247,0.936365,1.0,0.346355,12.965,24.740669,3.009622,0.150481,20.0
1,cancer,FACETIndex,False,Probability,100.0,NaN,4.5,0.947611,0.926247,0.936365,1.0,0.349135,13.400,16.149802,2.976366,0.148818,20.0
2,cancer,FACETIndex,False,Size,100.0,NaN,4.5,0.947611,0.926247,0.936365,1.0,0.353401,13.540,37.158960,3.073729,0.153686,20.0
3,cancer,FACETIndex,True,Axes,100.0,NaN,4.5,0.947611,0.926247,0.936365,1.0,0.349084,13.100,18.181202,3.079650,0.153983,20.0
4,cancer,FACETIndex,True,Probability,100.0,NaN,4.5,0.947611,0.926247,0.936365,1.0,0.351043,13.430,14.651625,3.037543,0.151877,20.0


In [14]:
all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]]

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
0,cancer,False,Axes,0.346355,12.965,24.740669
1,cancer,False,Probability,0.349135,13.400,16.149802
2,cancer,False,Size,0.353401,13.540,37.158960
3,cancer,True,Axes,0.349084,13.100,18.181202
4,cancer,True,Probability,0.351043,13.430,14.651625
5,cancer,True,Size,0.353947,13.565,30.551170
6,glass,False,Axes,0.163393,5.530,17.759281
7,glass,False,Probability,0.171278,5.970,8.805371
8,glass,False,Size,0.176913,6.135,30.974672
9,glass,True,Axes,0.163854,5.615,11.262533


In [24]:
idx = all_results.groupby(["explainer", "dataset"])["avg_dist"].idxmin()
all_results.loc[idx, ["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].head(100)

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
0,cancer,False,Axes,0.346355,12.965,24.740669
6,glass,False,Axes,0.163393,5.530,17.759281
12,magic,False,Axes,0.119507,8.475,25.379754
18,spambase,False,Axes,0.080537,17.185,34.011436
24,vertebral,False,Axes,0.094104,3.900,17.770938


In [23]:
idx = all_results.groupby(["explainer", "dataset", "hard_voting"])["avg_dist"].idxmin()
all_results.loc[idx, ["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time", "per_valid"]].head(100)

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time,per_valid
0,cancer,False,Axes,0.346355,12.965,24.740669,1.0
3,cancer,True,Axes,0.349084,13.100,18.181202,1.0
6,glass,False,Axes,0.163393,5.530,17.759281,1.0
9,glass,True,Axes,0.163854,5.615,11.262533,1.0
12,magic,False,Axes,0.119507,8.475,25.379754,1.0
15,magic,True,Axes,0.119863,8.515,18.507470,1.0
18,spambase,False,Axes,0.080537,17.185,34.011436,1.0
21,spambase,True,Axes,0.081188,17.295,27.415044,1.0
24,vertebral,False,Axes,0.094104,3.900,17.770938,1.0
27,vertebral,True,Axes,0.094311,3.940,11.252087,1.0


In [25]:
all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]]

,dataset,hard_voting,enumeration,avg_dist,avg_length,prep_time
0,cancer,False,Axes,0.346355,12.965,24.740669
1,cancer,False,Probability,0.349135,13.400,16.149802
2,cancer,False,Size,0.353401,13.540,37.158960
3,cancer,True,Axes,0.349084,13.100,18.181202
4,cancer,True,Probability,0.351043,13.430,14.651625
5,cancer,True,Size,0.353947,13.565,30.551170
6,glass,False,Axes,0.163393,5.530,17.759281
7,glass,False,Probability,0.171278,5.970,8.805371
8,glass,False,Size,0.176913,6.135,30.974672
9,glass,True,Axes,0.163854,5.615,11.262533


In [31]:
datasets = all_results["dataset"].unique()
votings = all_results["hard_voting"].unique()
enums = all_results["enumeration"].unique()

df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]]
df = df.pivot(index=["dataset", "hard_voting"], columns=["enumeration"], values=["avg_dist"])
# df = pd.DataFrame(, index=pd.MultiIndex.from_product([datasets, votings, enums]))
df.head(100)

avg_dist                      
enumeration                Axes Probability      Size
dataset   hard_voting                                
cancer    False        0.346355    0.349135  0.353401
          True         0.349084    0.351043  0.353947
glass     False        0.163393    0.171278  0.176913
          True         0.163854    0.173488  0.177412
magic     False        0.119507    0.122335  0.123277
          True         0.119863    0.122593  0.123350
spambase  False        0.080537    0.086237  0.088855
          True         0.081188    0.087559  0.089249
vertebral False        0.094104    0.095538  0.096155
          True         0.094311    0.095682  0.096358

In [19]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["dataset", "hard_voting", "enumeration"])
df.head()

avg_dist  avg_length  prep_time
dataset hard_voting enumeration                                 
cancer  False       Axes         0.346355      12.965  24.740669
                    Probability  0.349135      13.400  16.149802
                    Size         0.353401      13.540  37.158960
        True        Axes         0.349084      13.100  18.181202
                    Probability  0.351043      13.430  14.651625

In [20]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["hard_voting", "dataset", "enumeration"])
diff = df.loc[False,:,:]["avg_dist"] / df.loc[True,:,:]["avg_dist"]
diff.head(100)

dataset    enumeration
cancer     Axes           0.992180
           Probability    0.994566
           Size           0.998457
glass      Axes           0.997186
           Probability    0.987258
           Size           0.997186
magic      Axes           0.997029
           Probability    0.997894
           Size           0.999407
spambase   Axes           0.991988
           Probability    0.984908
           Size           0.995581
vertebral  Axes           0.997811
           Probability    0.998492
           Size           0.997896
Name: avg_dist, dtype: float64

In [21]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["hard_voting", "dataset", "enumeration"])
df = df.loc[False,:,:].reset_index()
mins = df.groupby(["dataset"])["avg_dist"].min()
df = df.set_index(["dataset", "enumeration"])["avg_dist"] / mins
df.head(100)
# df[["dataset", "enumeration", "avg_dist"]].groupby(["dataset", "enumeration"]) / mins
# df = all_results.groupby(["explainer", "dataset", "hard_voting"])["avg_dist"].min()
# all_results.loc[idx, ["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].head(100)

dataset    enumeration
cancer     Axes           1.000000
           Probability    1.008028
           Size           1.020346
glass      Axes           1.000000
           Probability    1.048257
           Size           1.082745
magic      Axes           1.000000
           Probability    1.023662
           Size           1.031544
spambase   Axes           1.000000
           Probability    1.070774
           Size           1.103274
vertebral  Axes           1.000000
           Probability    1.015234
           Size           1.021791
Name: avg_dist, dtype: float64

In [22]:
enum_comp = df.loc[False,:,:]["avg_dist"]
enum_comp.head(100)

KeyError: False

ds: cancer, enum: Axes, hard-soft: 0.003235322609509872
ds: cancer, enum: Probability, hard-soft: 0.00314414648596828
ds: cancer, enum: Size, hard-soft: 0.0004665221096812888
ds: glass, enum: Axes, hard-soft: 0.0003131748118953004
ds: glass, enum: Probability, hard-soft: 0.000944138563838709
ds: glass, enum: Size, hard-soft: 0.0
ds: magic, enum: Axes, hard-soft: 9.184607771628439e-05
ds: magic, enum: Probability, hard-soft: -5.2702417714711736e-05
ds: magic, enum: Size, hard-soft: 1.181401641051072e-05
ds: spambase, enum: Axes, hard-soft: 0.0006783338872494993
ds: spambase, enum: Probability, hard-soft: -0.00033281111304049593
ds: spambase, enum: Size, hard-soft: 0.00016610879487040375
ds: vertebral, enum: Axes, hard-soft: 3.4520391881801005e-05
ds: vertebral, enum: Probability, hard-soft: -0.0001515624791944975
ds: vertebral, enum: Size, hard-soft: 2.4676515291005563e-06


In [ ]:
df = all_results[["dataset","hard_voting", "enumeration", "avg_dist", "avg_length", "prep_time"]].set_index(["dataset", "enumeration", "hard_voting"])
for ds in all_results["dataset"].unique():
    for enum in all_results["enumeration"].unique():
        soft_dist = df.loc[ds, enum, False]["avg_dist"]
        hard_dist = df.loc[ds, enum, True]["avg_dist"]
        print("ds: {}, enum: {}, hard-soft: {}".format(ds, enum, hard_dist-soft_dist))

In [ ]:
cmap = plt.get_cmap("tab10")
colors = cmap.colors
ds_colors = {
    "cancer": colors[0],
    "glass": colors[1],
    "magic": colors[2],
    "spambase": colors[3],
    "vertebral": colors[4]
}
ds_markers = {
    "cancer": "^",
    "glass": "o",
    "magic": "v",
    "spambase": "s",
    "vertebral": "D"
}
marker_size = 8
line_width = 2
marker_sizes = {
    "cancer": marker_size,
    "glass": marker_size,
    "magic": marker_size,
    "spambase": marker_size,
    "vertebral": marker_size
}

In [ ]:
def render_plot(xfeature, yfeature, xlabel, ylabel, title, save_name=None, xrange=None, yrange=None, xlog=False, ylog=False, lloc=None, loc="best", lhorz=False):

    fig = plt.figure(figsize=[6.4, 4.8], frameon=True)
    ax = fig.add_subplot(1, 1, 1)
    for ds in all_results["dataset"].unique():
        is_ds = all_results["dataset"] == ds
        for expl in all_results["explainer"].unique():
            is_expl = all_results["explainer"] == expl
            matching_rows = is_ds & is_expl
            ax.plot(all_results[matching_rows][xfeature], all_results[matching_rows][yfeature], label=ds, lw=line_width, marker=ds_markers[ds], ms=marker_sizes[ds], color=ds_colors[ds])

        # title and axis labels
        if title is not None:
            plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        # legend
        if lhorz:
            plt.legend(loc=loc, ncol=len(all_results["dataset"].unique()), bbox_to_anchor=(0, 1.09, 1, 0), handletextpad=0.5, columnspacing=1, handlelength=1.5)
        else:
            plt.legend(loc=loc)
        # x and y ranges
        if xrange is not None:
            plt.xlim(xrange)
        if yrange is not None:
            plt.ylim(yrange)
        # x and y scales
        if ylog:
            plt.yscale('log')
        if xlog:
            plt.xscale('log')
        plt.subplots_adjust(top=1.0)
        
        if export_figures and save_name is not None:
            plt.savefig(output_dir + save_name, bbox_inches="tight")

In [ ]:
all_results["n_rects"].unique()

array([   100,   1000,   5000,  10000,  20000,  30000,  40000,  50000,
        60000,  70000,  80000,  90000, 100000], dtype=int64)

In [ ]:
render_plot(xfeature="n_rects", yfeature="avg_dist", xlabel="Number of Hyperrectangles", ylabel="Explanation Distance", title=None, save_name="dist_vs_rects.png", loc="upper center", lhorz=True)

In [ ]:
render_plot(xfeature="n_rects", yfeature="prep_time", xlabel="Number of Rectangles", ylabel="Initialization Time (sec)", title=None, save_name="prep_vs_rects.png", loc="upper center", lhorz=True)

In [ ]:
render_plot(xfeature="n_rects", yfeature="sample_time", xlabel="Number of Rectangles", ylabel="Sample Explanation Time (sec)", title=None, save_name="explain_vs_nrects", loc="upper center", lhorz=True)